In [1]:
import pandas as pd
from pandas import read_csv

from utils.configuration import Configuration

config = Configuration()

In [2]:
configs_mrr = pd.read_csv('../' + config.analyse_dir + '/mrr_with_adapt.csv')
configs_mrr.sort_values(by = ['MRR'], ascending=False, inplace=True)


In [3]:
def standard_expand(row):
    if row['training_set'] == 'standard':
        return row['training_set'] +'*'+ row['word_embedding']
    return row['training_set']

def change_columns_order(df):
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    return df[cols]

def coverage_finder(configs_mrr):
    covers = []
    for i in range(15):
        sample_config = configs_mrr.iloc[::i+1].copy()
        step = sample_config.agg(['nunique'])
        step['step'] = i+1
        covers.append(step)
    coverage = pd.concat(covers).reset_index(drop=True).sort_values(by = ['step'], ascending=True)
    coverage.rename(columns={'MRR':'n_config'}, inplace=True)
    return change_columns_order(coverage)

In [4]:

coverage = coverage_finder(configs_mrr)
coverage.to_csv('../' + config.analyse_dir + '/config_sample_stats.csv', index=False)
standard_expanded = configs_mrr.copy()
standard_expanded['training_set'] = configs_mrr.apply(standard_expand, axis=1)
coverage_standard = coverage_finder(standard_expanded)
coverage_standard.to_csv('../' + config.analyse_dir + '/config_sample_expanded_stats.csv', index=False)

In [5]:
configs_mrr.iloc[::5].to_csv('../' + config.analyse_dir + '/config_sample.csv', index=False)

In [6]:
configs_all = read_csv('../' + config.analyse_dir + '/mrr.csv').drop(columns=['top1'])
configs_sample = read_csv('../' + config.analyse_dir + '/config_sample.csv')

pd.concat([configs_all,configs_sample]).drop_duplicates(keep=False).to_csv('../' + config.analyse_dir + '/config_sample_new.csv', index=False)